<h3>Generally, I will take 3 steps:</h3>
<ul>
  <li>Pull data from the feature store</li>
  <li>Pull the model from the model registry</li>
  <li>Make predictions and create a feature group with patient_id, date, and prediction</li>
</ul>


In [1]:
import hopsworks
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()
hopswork_key = os.getenv("HOPSWORK_KEY")

In [3]:
project = hopsworks.login(api_key_value=hopswork_key)
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/112801
Connected. Call `.close()` to terminate connection gracefully.


In [4]:
feature_view  = fs.get_feature_view(
        name="diabetes_prediction_feature_view",
        version=1
    )

In [5]:
import datetime
start_date = (datetime.datetime.now() - datetime.timedelta(hours=24)) 
end_date = (datetime.datetime.now()) 

In [6]:
features_df = feature_view.get_batch_data(start_time = start_date, end_time = end_date)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.98s) 


In [7]:
features_df.shape

(98, 8)

In [8]:
features_df.head()

,patient_id,age,hypertension,heart_disease,bmi,hba1c_level,blood_glucose_level,datetime
0,49,74,0,0,30.00,6.0,126,2024-07-22 09:41:55.358864+00:00
1,30,32,0,0,37.68,3.5,144,2024-07-22 09:41:55.358864+00:00
2,29,79,0,0,37.62,6.6,218,2024-07-22 09:41:55.358864+00:00
3,97,36,0,0,38.80,5.8,128,2024-07-22 09:41:55.358864+00:00
4,22,33,0,0,30.58,5.8,127,2024-07-22 09:41:55.358864+00:00


<h3>Making prediction</h3>

In [9]:
import joblib

In [10]:
mr = project.get_model_registry()

Connected. Call `.close()` to terminate connection gracefully.


In [11]:
model = mr.get_model(name="diabetes_prediction_model", version=1)

In [12]:
model_dir = model.download()

In [13]:
model_path = os.path.join(model_dir, 'diab_pred_model.pkl')
loaded_model = joblib.load(model_path)

In [14]:
print("Contents of model_dir:", os.listdir(model_dir))

Contents of model_dir: ['input_example.json', 'diab_pred_model.pkl', 'model_schema.json']


In [15]:
features_df.head()

,patient_id,age,hypertension,heart_disease,bmi,hba1c_level,blood_glucose_level,datetime
0,49,74,0,0,30.00,6.0,126,2024-07-22 09:41:55.358864+00:00
1,30,32,0,0,37.68,3.5,144,2024-07-22 09:41:55.358864+00:00
2,29,79,0,0,37.62,6.6,218,2024-07-22 09:41:55.358864+00:00
3,97,36,0,0,38.80,5.8,128,2024-07-22 09:41:55.358864+00:00
4,22,33,0,0,30.58,5.8,127,2024-07-22 09:41:55.358864+00:00


In [16]:
primary_key_df = features_df[["patient_id","datetime"]]
features_df = features_df.drop(["patient_id","datetime"], axis=1)
features_df.head()

,age,hypertension,heart_disease,bmi,hba1c_level,blood_glucose_level
0,74,0,0,30.00,6.0,126
1,32,0,0,37.68,3.5,144
2,79,0,0,37.62,6.6,218
3,36,0,0,38.80,5.8,128
4,33,0,0,30.58,5.8,127


In [19]:
prediction = loaded_model.predict(features_df)

In [22]:
prediction_df = primary_key_df.copy()
prediction_df["prediction"] = prediction
prediction_df.head(3)

,patient_id,datetime,prediction
0,49,2024-07-22 09:41:55.358864+00:00,1
1,30,2024-07-22 09:41:55.358864+00:00,0
2,29,2024-07-22 09:41:55.358864+00:00,1


In [23]:
prediction_fg = fs.get_or_create_feature_group(
        name="diabetes_predictions",
        version=1,
        primary_key=['patient_id', 'datetime'],
        event_time='datetime',
        description="Predicted diabetes labels"
    )

    # Insert predictions into the new feature group
prediction_fg.insert(prediction_df)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/112801/fs/112720/fg/1025387


Uploading Dataframe: 0.00% |          | Rows 0/98 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: diabetes_predictions_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/112801/jobs/named/diabetes_predictions_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7f3f007ae8f0>, None)